In [150]:


# %%
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import sys
from sklearn.model_selection import train_test_split


def add3LIndicator_2D(change_p):
    if change_p >= 0.04:
        return 'V'
    elif change_p <= -0.04:
        return 'V'
    else:
        return 'NV'
    
def addLabel(value):
    if value == 'V':
        a = 1
    elif value == 'NV':
        a = 0
        
    return a

def add3LIndicator_3D(change_p):
    if change_p >= 0.04:
        return 'U'
    elif change_p <= -0.04:
        return 'D'
    else:
        return 'NV'
    
def addLabel_3D(value):
    if value == 'U':
        a = 1
    elif value == 'NV':
        a = 0
    else:
        a = 2
        
    return a


# %%
def label_(x, new_df, y):
    max_ = new_df.iloc[x]['max_ch']
    min_ = new_df.iloc[x]['min_ch']
    
#     if (max_ >= 0.12) & (min_ >= -0.06):
#         return 1
#     elif (max_ <= 0.06) & (min_ <= -0.12):
#         return 1
#     else:
#         return 0
    if (max_ >= y) :
        return 1
    elif (min_ <= -y):
        return 1
    else:
        return 0
    
def label_d(x, new_df, y):
    max_ = new_df.iloc[x]['max_ch']
    min_ = new_df.iloc[x]['min_ch']
    
#     if (max_ >= 0.12) & (min_ >= -0.06):
#         return 1
#     elif (max_ <= 0.06) & (min_ <= -0.12):
#         return 1
#     else:
#         return 0
    if (max_ >= y) :
        return 1
    elif (min_ <= -y):
        return 0
    else:
        return 2
    
def label_dd(x, new_df, y):
    max_ = new_df.iloc[x]['max_ch']
    min_ = new_df.iloc[x]['min_ch']
    
#     if (max_ >= 0.12) & (min_ >= -0.06):
#         return 1
#     elif (max_ <= 0.06) & (min_ <= -0.12):
#         return 1
#     else:
#         return 0
    if (min_ <= -y) :
        return 0
    elif (max_ >= y):
        return 1
    else:
        return 2
    
def calcEpsTTM(array):
    array = array.replace('[', '')
    array = array.replace(']', '')
    eps = [float(s) for s in array.split(', ')]
        
    return (eps[0]+eps[1]+eps[2]+eps[3])
    
def fundamental(symbol, tech_df):
    
    cal_df = pd.read_csv('fundamental/'+ symbol)
    cal_df = cal_df.set_index(['Unnamed: 0'])

    eps_list = []
    eps_act_list = []
    pre_eps_vec_list = []
    post_eps_vec_list = []

    for index, row in tech_df.iterrows():
        tech_df.loc[index, 'event'] = cal_df.loc[row['Date'], 'event']
        tech_df.loc[index, 'YoY'] = cal_df.loc[row['Date'], 'YoY']
        tech_df.loc[index, 'YoYRev'] = cal_df.loc[row['Date'], 'YoYRev']
        tech_df.loc[index, 'surprisePercent'] = cal_df.loc[row['Date'], 'surprisePercent']
        tech_df.loc[index, 'operatingMargin'] = cal_df.loc[row['Date'], 'operatingMargin']
        eps_list.append(cal_df.loc[row['Date'], 'eps_seq'])
        eps_act_list.append(cal_df.loc[row['Date'], 'eps_actual'])
        pre_eps_vec_list.append(cal_df.loc[row['Date'], 'pre_eps_vec'])
        post_eps_vec_list.append(cal_df.loc[row['Date'], 'post_eps_vec'])

    tech_df['eps_seq'] = eps_list
    tech_df['eps_actual'] = eps_act_list
    tech_df['pre_eps_vec'] = pre_eps_vec_list
    tech_df['post_eps_vec'] = post_eps_vec_list


    return tech_df


# %%
def calcEpsTTM(array):
    array = array.replace('[', '')
    array = array.replace(']', '')
    eps = [float(s) for s in array.split(', ')]
        
    return (eps[0]+eps[1]+eps[2]+eps[3])

def direction(x):
    if x >= 0.04:
        return 1
    elif x <= -0.04:
        return 0
    else:
        return 2
    
def vola(x):
    if x >= 0.04:
        return 1
    elif x <= -0.04:
        return 1
    else:
        return 0


# %%
# t = pd.DataFrame({'tickers' :tickers_})
# t.to_csv('chosen_stocls_direction_v2_70.csv',index = False)


# %%
# tickers = pd.read_csv('chosen_stocls_direction_v2_70.csv')['tickers'].unique().tolist()


# %%
# pd.read_csv('chosen_stocls_direction_v2_70.csv')['tickers'].apply(lambda x: x.split('.')[0]).unique().tolist()


# %%
# pd.read_csv('prod_up_tickers.csv')['tickers'].apply(lambda x: x.split('.')[0]).unique().tolist()





# %%
def direction(x):
    if x >= 0.04:
        return 1
    elif x <= -0.04:
        return 0
    else:
        return 2
    
def vola(x):
    if x >= 0.03:
        return 1
    elif x <= -0.03:
        return 1
    else:
        return 0


# %%
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense, Conv2D, Flatten, concatenate
from keras.layers.recurrent import LSTM
from keras.layers import Dropout, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense, Conv2D, Flatten, concatenate, BatchNormalization
from keras.layers.recurrent import LSTM
from keras.layers import Dropout, Bidirectional
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import keras.backend as K
import sys

def convToNumpyArr(s, n):
    str_arrays = s.split('], ')
    #arr = np.empty(shape=[0, n])
    arr = []
    
    for array in str_arrays:
        array = array.replace('[[', '')
        array = array.replace('[', '')
        array = array.replace(']', '')
        array = array.replace(']]', '')
        py_list = [float(s) for s in array.split(',')]
        arr.append(py_list)
    
    return arr

from keras import backend as K

def recall_m(y_true, y_pred):
    y_true= tf.cast(y_true, dtype='float64')
    y_pred= tf.cast(y_pred, dtype='float64')
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    y_true= tf.cast(y_true, dtype='float64')
    y_pred= tf.cast(y_pred, dtype='float64')
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    y_true= tf.cast(y_true, dtype='float64')
    y_pred= tf.cast(y_pred, dtype='float64')
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



def custom_loss(y_true, y_pred):
    # calculating squared difference between target and predicted values 
    y_true= tf.cast(y_true, dtype='float64')
    y_pred= tf.cast(y_pred, dtype='float64')
    loss = K.square(((y_pred - y_true)* [1.0, 3.0]))  # (batch_size, 2)
                
    # summing both loss values along batch dimension 
    loss = K.sum(loss, axis=1)        # (batch_size,)
    
    return loss

def add3LIndicator_2D(change_p):
    if change_p >= 0.04:
        return 'V'
    elif change_p <= -0.04:
        return 'V'
    else:
        return 'NV'
    
def addLabel(value):
    if value == 'V':
        a = 1
    elif value == 'NV':
        a = 0
        
    return a

def add3LIndicator_3D(change_p):
    if change_p >= 0.04:
        return 'U'
    elif change_p <= -0.04:
        return 'D'
    else:
        return 'NV'
    
def addLabel_3D(value):
    if value == 'U':
        a = 1
    elif value == 'NV':
        a = 0
    else:
        a = 2
        
    return a

def convToNumpyArrEPS(s, n):
    str_arrays = s.split('], ')
    #arr = np.empty(shape=[0, n])
    arr = []
    
    for array in str_arrays:
        array = array.replace('[[', '')
        array = array.replace('[', '')
        array = array.replace(']', '')
        array = array.replace(']]', '')
        py_list = [float(s) for s in array.split(', ')]
        arr.append(py_list)
    
    return arr


def convToNumpyArrEPSVEC(array):
    array = array.replace('[', '')
    array = array.replace(']', '')
    py_list = [float(s) for s in array.split(', ')]
    
    return py_list


# %%
a = 'aapl_df, amzn_df, bb_df, clf_df, clne_df, fb_df, ibm_df, msft_df, nok_df, nvda_df,shop_df, sndl_df, tlry_df, tsla_df, wsbc_df, jpm_df, jnj_df, unh_df, v_df, pg_df, hd_df, pypl_df, dis_df, bac_df, ma_df,cmcsa_df, pfe_df, crm_df, nflx_df, csco_df, xom_df, vz_df, abt_df, tmo_df, ko_df,tmus_df, unp_df, hon_df, ups_df, ms_df, amgn_df, pm_df, c_df, bmy_df,lin_df, low_df, ba_df, sbux_df, chtr_df, intu_df, now_df, schw_df, blk_df, amd_df,t_df, mmm_df, bkng_df, gs_df, rtx_df, amat_df, amt_df, de_df, zts_df, isrg_df, adi_df, pld_df, chtr_df, tgt_df, ge_df, antm_df, axp_df, cvs_df, spgi_df, cat_df,syk_df, mo_df, gild_df, lrcx_df, cop_df, lmt_df, tjx_df, adp_df, mdlz_df, mu_df, pnc_df, mmc_df, usb_df, cci_df, cb_df, eqix_df, gm_df, duk_df, fis_df'


# %%
tickers = []
for i in a.split(','):
    tickers.append(i.split('_')[0].upper().replace(' ','') + '.csv')





# %%
# import helper
import os
pd.options.mode.chained_assignment = None
# tickers = tickers + ['SPY.csv']
# tick = []

# # av = ['OGN.csv', 'BF.csv', 'BRK.csv', 'CARR.csv', 'OTIS.csv','BRK-A.csv','MRNA.csv','TSLA.csv','NVDA.csv']
# av = []
# li = []
# for i in os.listdir('/Volumes/AARTHI/sematic_data/lstm_training_data'):
# #     if (i != '.ipynb_checkpoints') and (i!= 'VOO.csv') and (i not in av):
#     if (i != '.ipynb_checkpoints') and (i not in av) :
#         li.append(i)

# # # # av = ['UWMC.csv','ANAC.csv','CRSR.csv','ARVL.csv','PLTR.csv','AGC.csv','LMAO.csv','OG.csv','U.csv','ACKIT.csv','CCIV.csv','RSI.csv','.DS_Store','ACND.csv','BREZ.csv','ACEVW.csv','BDTX.csv','PSFE.csv','RBLX.csv','ASO.csv','RKT.csv']
# # for i in os.listdir('fundamental'):
# # #     if (i != '.ipynb_checkpoints') and (i!= 'VOO.csv') and (i not in av):
# #     if (i != '.ipynb_checkpoints') and (i not in av) :
# #         tick.append(i)

# df_rate = pd.read_csv('Fed_rate.csv')
# spx = pd.read_csv('spx.csv')
# spx['Date'] = spx['Date'].astype(str)
# spx = spx[['Date','Adjusted_close']]
# spx.columns = ['Date','SNP_Close']
# cc = 0
# df_list = []
# avoid = []
# for i in tickers:
#     if (i in li) :
#         cc += 1
#         print(cc)
#         aa = i.split('.')[0]

#         temp = pd.read_csv('/Volumes/AARTHI/sematic_data/lstm_training_data/' + i)
#         temp['ticker'] = i
#         # df_list.append(temp)
#         if i != 'SPY.csv':

#             try:
#                 temp = helper.fundamental(aa, temp)
#                 if (temp['YoYRev'].isnull().sum() == 0) & (temp['operatingMargin'].isnull().sum() == 0):
#                     temp["EpsTTM"] = temp["eps_actual"].apply(calcEpsTTM)
#                     temp["EPRatio"]= temp["EpsTTM"] / temp["Adjusted_close"]

#                     fund_df = temp.merge(df_rate, on= ['Date']).drop(columns = ['Unnamed: 0'])
#                     fund_df = fund_df.merge(spx, on = 'Date')

#                     #Start with fundamental
#                     fund_df['Price_OED'] = None
#                     fund_df['SNP_Price_OED'] = None

#                     price_oed = None #Adjusted closing price on Earnings Day
#                     snp_price_oed = None #Adjusted closing price of S&P500 on Earnings Day
#                     prev_eps_seq = fund_df.iloc[0].eps_seq

#                     for i in range(fund_df.shape[0]):
#                         if (fund_df.iloc[i].event == 1.0):
#                             price_oed = fund_df.iloc[i].Adjusted_close
#                             snp_price_oed = fund_df.iloc[i].SNP_Close
#                             prev_eps_seq = fund_df.iloc[i].eps_seq
#                         elif (fund_df.iloc[i].eps_seq != prev_eps_seq):
#                             price_oed = fund_df.iloc[i].Adjusted_close
#                             snp_price_oed = fund_df.iloc[i].SNP_Close
#                             prev_eps_seq = fund_df.iloc[i].eps_seq
                            
#                         fund_df.iloc[i, fund_df.columns.get_loc('Price_OED')] = price_oed
#                         fund_df.iloc[i, fund_df.columns.get_loc('SNP_Price_OED')] = snp_price_oed

#                     fund_df["EpsTTM"] = fund_df["eps_actual"].apply(calcEpsTTM)
#                     fund_df["EPRatio"]= fund_df["EpsTTM"] / fund_df["Adjusted_close"]

#                     fund_df['Price_Ratio'] = fund_df['Adjusted_close'] / fund_df['Price_OED']
#                     fund_df['Price_Surprise_Ratio'] = fund_df['Price_Ratio'] / (1 + fund_df['surprisePercent'])

#                     fund_df['Adj_YoY'] = fund_df['YoY'] * abs(fund_df['YoY'])
#                     fund_df['Adj_surprisePercent'] = (1/3000) * fund_df['surprisePercent'] * abs(fund_df['surprisePercent'])

#                     fund_df['alt_surprise'] = (fund_df['epsDifference'] / fund_df['Adjusted_close']) * 100

#                     fund_df['alt_Price_Surprise_Ratio'] = fund_df['Price_Ratio'] / (1 + (fund_df['alt_surprise']/100))
#                     fund_df['SNP_Price_Ratio'] = fund_df['SNP_Close'] / fund_df['SNP_Price_OED']

#                     print('yes')

#                     df_list.append(fund_df)
#                 else:
#                     avoid.append(i)
#                     print(i)
#             except:
#                 print('Failed')
#                 avoid.append(i)
#                 print(i)
#         else:
#             spy = temp.copy()
#     else:
#         print('No Fundamentals - {}'.format(i))
#         avoid.append(i)

# print('Completed>>>>>>1')    
        
# df_test = []

# for i in tickers:
#     aa = i.split('.')[0]
#     if (i != '.DS_Store') & (i not in avoid) & (i in li) :

#         try:
#             temp = pd.read_csv('/Volumes/AARTHI/sematic_data/lstm_testing_data/' + i)
#             temp['ticker'] = i
#             # df_test.append(temp)

#             if i != 'SPY.csv':

#                 temp = helper.fundamental(aa, temp)
#                 if (temp['YoYRev'].isnull().sum() == 0) & (temp['operatingMargin'].isnull().sum() == 0):
#                     temp["EpsTTM"] = temp["eps_actual"].apply(calcEpsTTM)
#                     temp["EPRatio"]= temp["EpsTTM"] / temp["Adjusted_close"]

#                     fund_df = temp.merge(df_rate, on= ['Date']).drop(columns = ['Unnamed: 0'])
#                     fund_df = fund_df.merge(spx, on = 'Date')

#                     #Start with fundamental
#                     fund_df['Price_OED'] = None
#                     fund_df['SNP_Price_OED'] = None

#                     price_oed = None #Adjusted closing price on Earnings Day
#                     snp_price_oed = None #Adjusted closing price of S&P500 on Earnings Day
#                     prev_eps_seq = fund_df.iloc[0].eps_seq

#                     for i in range(fund_df.shape[0]):
#                         if (fund_df.iloc[i].event == 1.0):
#                             price_oed = fund_df.iloc[i].Adjusted_close
#                             snp_price_oed = fund_df.iloc[i].SNP_Close
#                             prev_eps_seq = fund_df.iloc[i].eps_seq
#                         elif (fund_df.iloc[i].eps_seq != prev_eps_seq):
#                             price_oed = fund_df.iloc[i].Adjusted_close
#                             snp_price_oed = fund_df.iloc[i].SNP_Close
#                             prev_eps_seq = fund_df.iloc[i].eps_seq
                            
#                         fund_df.iloc[i, fund_df.columns.get_loc('Price_OED')] = price_oed
#                         fund_df.iloc[i, fund_df.columns.get_loc('SNP_Price_OED')] = snp_price_oed

#                     fund_df["EpsTTM"] = fund_df["eps_actual"].apply(calcEpsTTM)
#                     fund_df["EPRatio"]= fund_df["EpsTTM"] / fund_df["Adjusted_close"]

#                     fund_df['Price_Ratio'] = fund_df['Adjusted_close'] / fund_df['Price_OED']
#                     fund_df['Price_Surprise_Ratio'] = fund_df['Price_Ratio'] / (1 + fund_df['surprisePercent'])

#                     fund_df['Adj_YoY'] = fund_df['YoY'] * abs(fund_df['YoY'])
#                     fund_df['Adj_surprisePercent'] = (1/3000) * fund_df['surprisePercent'] * abs(fund_df['surprisePercent'])

#                     fund_df['alt_surprise'] = (fund_df['epsDifference'] / fund_df['Adjusted_close']) * 100

#                     fund_df['alt_Price_Surprise_Ratio'] = fund_df['Price_Ratio'] / (1 + (fund_df['alt_surprise']/100))
#                     fund_df['SNP_Price_Ratio'] = fund_df['SNP_Close'] / fund_df['SNP_Price_OED']

#                     print('yes')

#                     df_test.append(fund_df)
#                 else:
#                     avoid.append(i)
#                     print(i)
#             else:
#                 spy_test = temp.copy()

#         except:
#             avoid.append(i)
#             print(i)

#         # temp["EpsTTM"] = temp["eps_actual"].apply(calcEpsTTM)
#         # temp["EPRatio"]= temp["EpsTTM"] / temp["Adjusted_close"]
#         # df_test.append(temp)




# print('Completed>>>>>>2')    
        




# new_df = pd.concat(df_list, ignore_index=True, sort=True)

# new_df = new_df.reset_index(drop = True)

# new_df = new_df.drop_duplicates().reset_index(drop = True)


# test_df = pd.concat(df_test, ignore_index=True, sort=True)
# test_df = test_df.reset_index(drop = True)

# print(len(new_df[['eps_seq',"pre_eps_vec","post_eps_vec",'epsDifference','YoY']]))

# new_df.to_csv('/Volumes/AARTHI/sematic_data/lstm_train.csv',index = False)
# test_df.to_csv('/Volumes/AARTHI/sematic_data/lstm_test.csv',index = False)

# spy.to_csv('/Volumes/AARTHI/sematic_data/spy_train.csv',index = False)
# spy_test.to_csv('/Volumes/AARTHI/sematic_data/spy_test.csv',index = False)

# new_df= pd.read_csv('/Volumes/AARTHI/sematic_data/lstm_train.csv')
test_df = pd.read_csv('/Volumes/AARTHI/sematic_data/lstm_test.csv')

spy = pd.read_csv('/Volumes/AARTHI/sematic_data/spy_train.csv')
spy_test = pd.read_csv('/Volumes/AARTHI/sematic_data/spy_test.csv')

test_df = test_df.drop_duplicates().reset_index(drop = True)

# new_df['vol'] = new_df['future_ch_p'].apply(vola)

# test_df['vol'] = test_df['future_ch_p'].apply(vola)



new_df = new_df.dropna().reset_index(drop = True)
test_df = test_df.dropna().reset_index(drop = True)



# new_df = new_df[(new_df['Date'] <= '2020-02-01') | ((new_df['Date'] >= '2020-08-01') & (new_df['Date'] < '2021-07-01')) ].reset_index(drop = True)
new_df = new_df[(new_df['Date'] <= '2019-12-01') ].reset_index(drop = True)



# len(test_df)
# test_df = test_df[((test_df['Date'] >= '2021-07-01') &(test_df['Date'] <= '2021-12-01')) | ((test_df['Date'] >= '2019-09-01') &(test_df['Date'] <= '2019-12-31'))].reset_index(drop = True)
test_df = test_df[((test_df['Date'] >= '2021-07-01') & (test_df['Date'] < '2021-10-01')  )| ((test_df['Date'] < '2020-02-01') & (test_df['Date'] > '2019-12-01'))].reset_index(drop = True)
# len(test_df)

def snpSeq(snp_df, date):
    try:

        return snp_df.loc[snp_df['Date'] == date].Change_Seq.tolist()[0]
    except:


        return np.nan

def snpFut5Price(snp_df, date, column_name):
    try:

        return float(snp_df.loc[snp_df['Date'] == date].future_ch_p.tolist()[0])
    except:
        

        return np.nan


new_df['SNP_P_Ch_Seq'] = [snpSeq(spy, date) for date in new_df['Date']]
test_df['SNP_P_Ch_Seq'] = [snpSeq(spy_test, date) for date in test_df['Date']]

new_df["SNP_5d_ch_p"] = [snpFut5Price(spy, date, "future_ch_p") for date in new_df['Date']]
new_df = new_df.dropna(subset = ["SNP_5d_ch_p"])
new_df = new_df.dropna(subset = ["SNP_P_Ch_Seq"])
new_df["fut_snp_5d_ch_p"] = new_df["future_ch_p"] - new_df["SNP_5d_ch_p"]

test_df["SNP_5d_ch_p"] = [snpFut5Price(spy_test, date, "future_ch_p") for date in test_df['Date']]
test_df = test_df.dropna(subset = ["SNP_5d_ch_p"])
test_df = test_df.dropna(subset = ["SNP_P_Ch_Seq"])
test_df["fut_snp_5d_ch_p"] = test_df["future_ch_p"] - test_df["SNP_5d_ch_p"]

new_df['vol'] = new_df["fut_snp_5d_ch_p"].apply(vola)
test_df['vol'] = test_df["fut_snp_5d_ch_p"].apply(vola)

new_df = new_df.sample(frac = 1)

new_df = new_df.dropna().reset_index(drop = True)


train_len = len(new_df)

new_df = new_df.append(test_df)



new_df = new_df.reset_index(drop = True)

# price_df = new_df["Price_Seq"]
snp_price_df = new_df["SNP_P_Ch_Seq"]
volume_df = new_df["Volume_Seq"]
change_df = new_df['Change_Seq']
eps_df = new_df["eps_seq"]
pre_eps_vec_df = new_df["pre_eps_vec"]
post_eps_vec_df = new_df["post_eps_vec"]

# price_lstm_ = []
snp_price_lstm_ = []
volume_lstm_ = []
pattern_lstm_ = []
eps_lstm_ = []
pre_eps_vec_lstm_ = []
post_eps_vec_lstm_ = []

print('completed------1 \n')

for i in range(change_df.shape[0]):
    # price_lstm_.append(convToNumpyArr(price_df.iloc[i], 4))
    snp_price_lstm_.append(convToNumpyArr(snp_price_df.iloc[i], 4))
    volume_lstm_.append(convToNumpyArr(volume_df.iloc[i], 2))
    pattern_lstm_.append(convToNumpyArr(change_df.iloc[i],4))
    eps_lstm_.append(convToNumpyArrEPS(eps_df.iloc[i], 2))
    pre_eps_vec_lstm_.append(convToNumpyArrEPSVEC(pre_eps_vec_df.iloc[i]))
    post_eps_vec_lstm_.append(convToNumpyArrEPSVEC(post_eps_vec_df.iloc[i]))
    
# price_lstm_np = np.array(price_lstm_)
# snp_price_lstm_np = np.array(snp_price_lstm_)
volume_lstm_np = np.array(volume_lstm_)
pattern_lstm_np = np.array(pattern_lstm_)
snp_price_lstm_np = np.array(snp_price_lstm_)
eps_lstm_np = np.array(eps_lstm_)
pre_eps_vec_lstm_np = np.array(pre_eps_vec_lstm_)
post_eps_vec_lstm_np = np.array(post_eps_vec_lstm_)

pattern_lstm_np = pattern_lstm_np - snp_price_lstm_np

new_df['Adj_YoY'] = new_df['YoY'] * new_df['YoY']
new_df['Adj_surprise'] = abs(new_df['epsDifference'] / new_df['Adjusted_close']) * 100
new_df['Adj_YoYRev'] = abs(new_df['YoYRev']) * 10

eps_vec_np = np.stack((pre_eps_vec_lstm_np, post_eps_vec_lstm_np), axis=2)

# test_price_lstm_np = price_lstm_np[train_len:]
# price_lstm_np = price_lstm_np[:train_len]

test_volume_lstm_np = volume_lstm_np[train_len:]
volume_lstm_np = volume_lstm_np[:train_len]

test_pattern_lstm_np = pattern_lstm_np[train_len:]
pattern_lstm_np = pattern_lstm_np[:train_len]

test_snp_price_lstm_np = snp_price_lstm_np[train_len:]
volume_lstm_np = volume_lstm_np[:train_len]

test_eps_lstm_np = eps_lstm_np[train_len:]
eps_lstm_np = eps_lstm_np[:train_len]

test_eps_vec_np = eps_vec_np[train_len:]
eps_vec_np = eps_vec_np[:train_len]

from keras.utils import np_utils

label_df = new_df["vol"][:train_len]
test_label_df = new_df["vol"][train_len:]
label_np = label_df.to_numpy()
test_label_np = test_label_df.to_numpy()
label_np_oh = np_utils.to_categorical(label_np)
test_np_oh = np_utils.to_categorical(test_label_np)

print('completed------2 \n')

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='min')
mcp_save = ModelCheckpoint('weigts_vol.h5', save_best_only=True, monitor='val_accuracy', mode='min')

X_train0_df = new_df[['Adj_YoY', 'Adj_surprise', 'Adj_YoYRev']]
X_train1_df =1 -  (new_df[['RSI9', 'RSI14', 'RSI20']] - 50)/50
X_train2_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p']] * 10
X_train3_df = new_df[['1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p']] * 10
#     X_train3_df = inference_df[['1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p',
#                          '1d_adx9_ch_p', '2d_adx9_ch_p', '3d_adx9_ch_p', '7d_adx9_ch_p',
#                           '1d_adx14_ch_p', '2d_adx14_ch_p', '3d_adx14_ch_p', '7d_adx14_ch_p',
#                          '1d_RSI14_ch_p', '2d_RSI14_ch_p', '3d_RSI14_ch_p', '7d_RSI14_ch_p']] * 10
#     X_train4_df = (inference_df[['ADX5', 'ADX9', 'ADX14', 'ADX20']] - 50)/50

#     X_train_df = pd.concat([X_train1_df, X_train2_df, X_train3_df,X_train4_df], axis=1)



# X_train1_df = (new_df[['RSI9', 'RSI14', 'RSI20']] - 50)/50
# X_train2_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p']] * 10
# X_train3_df = new_df[['1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p']] * 10
# X_train3_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p','1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p'
#                      ,"VWMA2_ch_p","VWMA5_ch_p","VWMA30_ch_p",'1d_ch_v','7d_ch_v','12d_ch_v','21d_ch_v',
#        'vola5','vola9','vola21','macd259','signal259','hist259','macd91121','signal91121','hist91121','alt_surprise', '2d_ch_p','VWMA20_ch_p', 'EPRatio','SNP_Price_Ratio','FR_1M_ch_p', 'operatingMargin','stoch_d','FR_3M_ch_p','bop']]

X_train_df = pd.concat([X_train0_df[:train_len], X_train1_df[:train_len],  X_train2_df[:train_len], X_train3_df[:train_len]], axis=1)
X_test_df = pd.concat([X_train0_df[train_len:], X_train1_df[train_len:],  X_train2_df[train_len:], X_train3_df[train_len:]], axis=1)

X_train_np = np.array(X_train_df)
X_test_np = np.array(X_test_df)

# define two sets of inputs

lstm_seq_size = 40


# price = Input(shape=(lstm_seq_size,4))
volume = Input(shape=(lstm_seq_size,2))
pattern = Input(shape=(lstm_seq_size,4))
technicals = Input(shape=(13,))
eps = Input(shape=(5,2))
eps_vec = Input(shape=(21,2))

print('completed------3 \n')

# the first branch operates on the first input - price
x = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(pattern)
x = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(x)
x = Dropout(0.1)(x)
x = Model(inputs=pattern, outputs=x)
# the second branch opreates on the second input - volume
y = LSTM(50,input_shape = (lstm_seq_size,2),return_sequences = True, activation = 'relu')(volume)
y = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(y)
y = Dropout(0.1)(y)
y = Model(inputs=volume, outputs=y)
# the third branch opreates on the third input - technicals
z = Dense(60, activation='relu', kernel_initializer='he_normal', input_shape=(13,))(technicals)
z = Dense(30, activation='relu')(z)
z = Dropout(0.1)(z)
z = Model(inputs=technicals, outputs=z)
# the fourth branch opreates on the second input - volume
l = LSTM(6,input_shape = (5,2),return_sequences = True, activation = 'relu')(eps)
l = Bidirectional(LSTM(10,return_sequences = False, activation = 'relu'))(l)
l = Dropout(0.1)(l)
l = Model(inputs=eps, outputs=l)
# the fifth branch opreates on the second input - volume
m = LSTM(3,input_shape = (21,2),return_sequences = True, activation = 'relu')(eps_vec)
m = Bidirectional(LSTM(5,return_sequences = False, activation = 'relu'))(m)
m = Model(inputs=eps_vec, outputs=m)
# combine the output of the two branches
combined = concatenate([x.output, y.output, z.output, l.output, m.output])
#combined = concatenate([x.output, y.output, z.output])
combined = Dropout(0.1)(combined)
#combined = Dense(40, activation='relu')(combined)
#combined = Dropout(0.1)(combined)
#combined = Multiply()([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
out = Dense(2, activation='softmax')(combined) #switched from 3
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input, z.input, l.input, m.input], outputs=out)



print('completed------4 \n')


# #encoder - used for our dimention reduction
# encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)
# # encoder = Model(inputs=[ y.input, z.input, p.input], outputs=combined)
# loss_fn = tf.keras.losses.CategoricalCrossentropy()
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# compile the model
# model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn, metrics=['accuracy'])

# compile the model
# model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.003), loss = custom_loss, metrics=[f1_score, 'accuracy', precision_m])
# model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.0005), loss = custom_loss, metrics=[precision_m, f1_score, 'accuracy'])



# print(model.summary())


# # fit the model
# # model.fit([pattern_lstm_np, volume_lstm_np, X_train_np ], label_np, epochs= 3, batch_size=256, validation_split = 0.2, callbacks=[earlyStopping, mcp_save], verbose=1)
# model.fit([pattern_lstm_np, volume_lstm_np, X_train_np, eps_lstm_np, eps_vec_np], label_np_oh, epochs=5, batch_size=64, validation_split = 0.15, callbacks=[earlyStopping, mcp_save], verbose=1)
# # print(model.summary())
# # model.evaluate([test_pattern_lstm_np,test_volume_lstm_np, X_test_np,test_eps_lstm_np, test_eps_vec_np ], test_label_np)
model.load_weights('/Users/kukeshayanth/Downloads/vol-100comp-snp-closs.h5')

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np,test_eps_lstm_np, test_eps_vec_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

completed------1 

completed------2 

completed------3 

The following Variables were used a Lambda layer's call (tf.keras.backend.rnn_288), but
are not present in its tracked objects:
  <tf.Variable 'lstm_193/lstm_cell_385/kernel:0' shape=(4, 200) dtype=float32>
  <tf.Variable 'lstm_193/lstm_cell_385/recurrent_kernel:0' shape=(50, 200) dtype=float32>
  <tf.Variable 'lstm_193/lstm_cell_385/bias:0' shape=(200,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.keras.backend.rnn_289), but
are not present in its tracked objects:
  <tf.Variable 'bidirectional_96/forward_lstm_194/lstm_cell_387/kernel:0' shape=(50, 240) dtype=float32>
  <tf.Variable 'bidirectional_96/forward_lstm_194/lstm_cell_387/recurrent_kernel:0' shape=(60, 240) dtype=float32>
  <tf.Variable 'bidirectio

In [174]:

test_df_copy = pd.read_csv('/Volumes/AARTHI/sematic_data/lstm_test.csv')

test_df =test_df_copy.copy()

spy_test = pd.read_csv('/Volumes/AARTHI/sematic_data/spy_test.csv')

test_df = test_df.drop_duplicates().reset_index(drop = True)

test_df = test_df.dropna().reset_index(drop = True)

test_df = test_df[((test_df['Date'] >= '2021-07-01') & (test_df['Date'] < '2021-10-01')  )| ((test_df['Date'] < '2020-02-01') & (test_df['Date'] > '2019-12-01'))].reset_index(drop = True)

def snpSeq(snp_df, date):
    try:

        return snp_df.loc[snp_df['Date'] == date].Change_Seq.tolist()[0]
    except:


        return np.nan

def snpFut5Price(snp_df, date, column_name):
    try:

        return float(snp_df.loc[snp_df['Date'] == date].future_ch_p.tolist()[0])
    except:
        

        return np.nan


test_df['SNP_P_Ch_Seq'] = [snpSeq(spy_test, date) for date in test_df['Date']]


test_df["SNP_5d_ch_p"] = [snpFut5Price(spy_test, date, "future_ch_p") for date in test_df['Date']]
test_df = test_df.dropna(subset = ["SNP_5d_ch_p"])
test_df = test_df.dropna(subset = ["SNP_P_Ch_Seq"])
test_df["fut_snp_5d_ch_p"] = test_df["future_ch_p"] - test_df["SNP_5d_ch_p"]

test_df['vol'] = test_df["fut_snp_5d_ch_p"].apply(vola)





new_df = test_df.copy()



new_df = new_df.reset_index(drop = True)

# price_df = new_df["Price_Seq"]
snp_price_df = new_df["SNP_P_Ch_Seq"]
volume_df = new_df["Volume_Seq"]
change_df = new_df['Change_Seq']
eps_df = new_df["eps_seq"]
pre_eps_vec_df = new_df["pre_eps_vec"]
post_eps_vec_df = new_df["post_eps_vec"]

# price_lstm_ = []
snp_price_lstm_ = []
volume_lstm_ = []
pattern_lstm_ = []
eps_lstm_ = []
pre_eps_vec_lstm_ = []
post_eps_vec_lstm_ = []

print('completed------1 \n')

for i in range(change_df.shape[0]):
    # price_lstm_.append(convToNumpyArr(price_df.iloc[i], 4))
    snp_price_lstm_.append(convToNumpyArr(snp_price_df.iloc[i], 4))
    volume_lstm_.append(convToNumpyArr(volume_df.iloc[i], 2))
    pattern_lstm_.append(convToNumpyArr(change_df.iloc[i],4))
    eps_lstm_.append(convToNumpyArrEPS(eps_df.iloc[i], 2))
    pre_eps_vec_lstm_.append(convToNumpyArrEPSVEC(pre_eps_vec_df.iloc[i]))
    post_eps_vec_lstm_.append(convToNumpyArrEPSVEC(post_eps_vec_df.iloc[i]))
    
# price_lstm_np = np.array(price_lstm_)
# snp_price_lstm_np = np.array(snp_price_lstm_)
volume_lstm_np = np.array(volume_lstm_)
pattern_lstm_np = np.array(pattern_lstm_)
snp_price_lstm_np = np.array(snp_price_lstm_)
eps_lstm_np = np.array(eps_lstm_)
pre_eps_vec_lstm_np = np.array(pre_eps_vec_lstm_)
post_eps_vec_lstm_np = np.array(post_eps_vec_lstm_)

pattern_lstm_np = pattern_lstm_np - snp_price_lstm_np

new_df['Adj_YoY'] = new_df['YoY'] * new_df['YoY']
new_df['Adj_surprise'] = abs(new_df['epsDifference'] / new_df['Adjusted_close']) * 100
new_df['Adj_YoYRev'] = abs(new_df['YoYRev']) * 10

eps_vec_np = np.stack((pre_eps_vec_lstm_np, post_eps_vec_lstm_np), axis=2)

# test_price_lstm_np = price_lstm_np[train_len:]
# price_lstm_np = price_lstm_np[:train_len]

test_volume_lstm_np = volume_lstm_np


test_pattern_lstm_np = pattern_lstm_np

test_snp_price_lstm_np = snp_price_lstm_np

test_eps_lstm_np = eps_lstm_np

test_eps_vec_np = eps_vec_np

from keras.utils import np_utils


test_label_df = new_df["vol"]

test_label_np = test_label_df.to_numpy()

test_np_oh = np_utils.to_categorical(test_label_np)

print('completed------2 \n')



X_train0_df = new_df[['Adj_YoY', 'Adj_surprise', 'Adj_YoYRev']]
X_train1_df =1 -  (new_df[['RSI9', 'RSI14', 'RSI20']] - 50)/50
X_train2_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p']] * 10
X_train3_df = new_df[['1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p']] * 10

X_test_df = pd.concat([X_train0_df, X_train1_df,  X_train2_df, X_train3_df], axis=1)


X_test_np = np.array(X_test_df)

# define two sets of inputs

lstm_seq_size = 40


# price = Input(shape=(lstm_seq_size,4))
volume = Input(shape=(lstm_seq_size,2))
pattern = Input(shape=(lstm_seq_size,4))
technicals = Input(shape=(13,))
eps = Input(shape=(5,2))
eps_vec = Input(shape=(21,2))

print('completed------3 \n')

# the first branch operates on the first input - price
x = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(pattern)
x = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(x)
x = Dropout(0.1)(x)
x = Model(inputs=pattern, outputs=x)
# the second branch opreates on the second input - volume
y = LSTM(50,input_shape = (lstm_seq_size,2),return_sequences = True, activation = 'relu')(volume)
y = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(y)
y = Dropout(0.1)(y)
y = Model(inputs=volume, outputs=y)
# the third branch opreates on the third input - technicals
z = Dense(60, activation='relu', kernel_initializer='he_normal', input_shape=(13,))(technicals)
z = Dense(30, activation='relu')(z)
z = Dropout(0.1)(z)
z = Model(inputs=technicals, outputs=z)
# the fourth branch opreates on the second input - volume
l = LSTM(6,input_shape = (5,2),return_sequences = True, activation = 'relu')(eps)
l = Bidirectional(LSTM(10,return_sequences = False, activation = 'relu'))(l)
l = Dropout(0.1)(l)
l = Model(inputs=eps, outputs=l)
# the fifth branch opreates on the second input - volume
m = LSTM(3,input_shape = (21,2),return_sequences = True, activation = 'relu')(eps_vec)
m = Bidirectional(LSTM(5,return_sequences = False, activation = 'relu'))(m)
m = Model(inputs=eps_vec, outputs=m)
# combine the output of the two branches
combined = concatenate([x.output, y.output, z.output, l.output, m.output])
#combined = concatenate([x.output, y.output, z.output])
combined = Dropout(0.1)(combined)
#combined = Dense(40, activation='relu')(combined)

out = Dense(2, activation='softmax')(combined) #switched from 3

model = Model(inputs=[x.input, y.input, z.input, l.input, m.input], outputs=out)



print('completed------4 \n')


model.load_weights('/Users/kukeshayanth/Downloads/vol-100comp-snp-closs.h5')

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np,test_eps_lstm_np, test_eps_vec_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

completed------1 

completed------2 

completed------3 

The following Variables were used a Lambda layer's call (tf.keras.backend.rnn_360), but
are not present in its tracked objects:
  <tf.Variable 'lstm_241/lstm_cell_481/kernel:0' shape=(4, 200) dtype=float32>
  <tf.Variable 'lstm_241/lstm_cell_481/recurrent_kernel:0' shape=(50, 200) dtype=float32>
  <tf.Variable 'lstm_241/lstm_cell_481/bias:0' shape=(200,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.keras.backend.rnn_361), but
are not present in its tracked objects:
  <tf.Variable 'bidirectional_120/forward_lstm_242/lstm_cell_483/kernel:0' shape=(50, 240) dtype=float32>
  <tf.Variable 'bidirectional_120/forward_lstm_242/lstm_cell_483/recurrent_kernel:0' shape=(60, 240) dtype=float32>
  <tf.Variable 'bidirect

In [176]:
volume = Input(shape=(lstm_seq_size,2))
pattern = Input(shape=(lstm_seq_size,4))
technicals = Input(shape=(13,))
eps = Input(shape=(5,2))
eps_vec = Input(shape=(21,2))

print('completed------3 \n')

# the first branch operates on the first input - price
x = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(pattern)
x = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(x)
x = Dropout(0.1)(x)
x = Model(inputs=pattern, outputs=x)
# the second branch opreates on the second input - volume
y = LSTM(50,input_shape = (lstm_seq_size,2),return_sequences = True, activation = 'relu')(volume)
y = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(y)
y = Dropout(0.1)(y)
y = Model(inputs=volume, outputs=y)
# the third branch opreates on the third input - technicals
z = Dense(60, activation='relu', kernel_initializer='he_normal', input_shape=(13,))(technicals)
z = Dense(30, activation='relu')(z)
z = Dropout(0.1)(z)
z = Model(inputs=technicals, outputs=z)
# the fourth branch opreates on the second input - volume
l = LSTM(6,input_shape = (5,2),return_sequences = True, activation = 'relu')(eps)
l = Bidirectional(LSTM(10,return_sequences = False, activation = 'relu'))(l)
l = Dropout(0.1)(l)
l = Model(inputs=eps, outputs=l)
# the fifth branch opreates on the second input - volume
m = LSTM(3,input_shape = (21,2),return_sequences = True, activation = 'relu')(eps_vec)
m = Bidirectional(LSTM(5,return_sequences = False, activation = 'relu'))(m)
m = Model(inputs=eps_vec, outputs=m)
# combine the output of the two branches
combined = concatenate([x.output, y.output, z.output, l.output, m.output])
#combined = concatenate([x.output, y.output, z.output])
combined = Dropout(0.1)(combined)
#combined = Dense(40, activation='relu')(combined)

out = Dense(2, activation='softmax')(combined) #switched from 3

model = Model(inputs=[x.input, y.input, z.input, l.input, m.input], outputs=out)



print('completed------4 \n')


model.load_weights('/Users/kukeshayanth/Downloads/vol-100comp-snp-closs.h5')

p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np,test_eps_lstm_np, test_eps_vec_np])



completed------3 

The following Variables were used a Lambda layer's call (tf.keras.backend.rnn_372), but
are not present in its tracked objects:
  <tf.Variable 'lstm_249/lstm_cell_497/kernel:0' shape=(4, 200) dtype=float32>
  <tf.Variable 'lstm_249/lstm_cell_497/recurrent_kernel:0' shape=(50, 200) dtype=float32>
  <tf.Variable 'lstm_249/lstm_cell_497/bias:0' shape=(200,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.keras.backend.rnn_373), but
are not present in its tracked objects:
  <tf.Variable 'bidirectional_124/forward_lstm_250/lstm_cell_499/kernel:0' shape=(50, 240) dtype=float32>
  <tf.Variable 'bidirectional_124/forward_lstm_250/lstm_cell_499/recurrent_kernel:0' shape=(60, 240) dtype=float32>
  <tf.Variable 'bidirectional_124/forward_lstm_250/lstm_cell_4

In [178]:
X_test_df

,Adj_YoY,Adj_surprise,Adj_YoYRev,RSI9,RSI14,RSI20,VWMA9_ch_p,VWMA14_ch_p,VWMA20_ch_p,1d_ch_p,2d_ch_p,3d_ch_p,7d_ch_p
0,0.000319,0.076836,0.181240,0.879633,0.764723,0.713634,-0.011236,-0.010638,0.055119,-0.115623,-0.137390,-0.004915,0.036863
1,0.000319,0.078231,0.181240,1.118191,0.940037,0.841139,-0.155755,-0.177886,-0.129196,-0.178306,-0.291868,-0.313247,-0.097717
2,0.000319,0.077547,0.181240,1.001552,0.875018,0.800532,-0.064785,-0.085227,-0.051724,0.088260,-0.091620,-0.206183,-0.001535
3,0.000319,0.076425,0.181240,0.836861,0.777853,0.737667,0.063310,0.052715,0.079083,0.146719,0.236274,0.053755,-0.029657
4,0.000319,0.074977,0.181240,0.671037,0.670709,0.664312,0.213946,0.229893,0.248178,0.193158,0.342710,0.433996,0.242914
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8886,0.066784,0.047283,1.731938,1.283501,1.273347,1.260870,-0.105729,-0.194844,-0.259730,-0.022893,-0.003277,0.053551,-0.245404
8887,0.066784,0.046412,1.731938,1.029313,1.122194,1.160953,0.073731,-0.005183,-0.066509,0.187659,0.164337,0.184320,-0.086124
8888,0.066784,0.047202,1.731938,1.192686,1.213516,1.220047,-0.062234,-0.117377,-0.218323,-0.167310,0.017209,-0.005723,-0.120424
8889,0.066784,0.046781,1.731938,1.084128,1.145636,1.174018,0.049192,-0.011015,-0.111933,0.089987,-0.078829,0.107351,0.017869


In [169]:
X_test_df

,Adj_YoY,Adj_surprise,Adj_YoYRev,RSI9,RSI14,RSI20,VWMA9_ch_p,VWMA14_ch_p,VWMA20_ch_p,1d_ch_p,2d_ch_p,3d_ch_p,7d_ch_p
0,0.000319,0.076836,0.181240,0.879633,0.764723,0.713634,-0.011236,-0.010638,0.055119,-0.115623,-0.137390,-0.004915,0.036863
1,0.000319,0.078231,0.181240,1.118191,0.940037,0.841139,-0.155755,-0.177886,-0.129196,-0.178306,-0.291868,-0.313247,-0.097717
2,0.000319,0.077547,0.181240,1.001552,0.875018,0.800532,-0.064785,-0.085227,-0.051724,0.088260,-0.091620,-0.206183,-0.001535
3,0.000319,0.076425,0.181240,0.836861,0.777853,0.737667,0.063310,0.052715,0.079083,0.146719,0.236274,0.053755,-0.029657
4,0.000319,0.074977,0.181240,0.671037,0.670709,0.664312,0.213946,0.229893,0.248178,0.193158,0.342710,0.433996,0.242914
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8886,0.066784,0.047283,1.731938,1.283501,1.273347,1.260870,-0.105729,-0.194844,-0.259730,-0.022893,-0.003277,0.053551,-0.245404
8887,0.066784,0.046412,1.731938,1.029313,1.122194,1.160953,0.073731,-0.005183,-0.066509,0.187659,0.164337,0.184320,-0.086124
8888,0.066784,0.047202,1.731938,1.192686,1.213516,1.220047,-0.062234,-0.117377,-0.218323,-0.167310,0.017209,-0.005723,-0.120424
8889,0.066784,0.046781,1.731938,1.084128,1.145636,1.174018,0.049192,-0.011015,-0.111933,0.089987,-0.078829,0.107351,0.017869


In [177]:
o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

[[7062    0]
 [1829    0]]
              precision    recall  f1-score   support

           0       0.79      1.00      0.89      7062
           1       0.00      0.00      0.00      1829

    accuracy                           0.79      8891
   macro avg       0.40      0.50      0.44      8891
weighted avg       0.63      0.79      0.70      8891



/Users/kukeshayanth/Desktop/semantic_corp/technical_analysis/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kukeshayanth/Desktop/semantic_corp/technical_analysis/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kukeshayanth/Desktop/semantic_corp/technical_analysis/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

In [175]:
o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

[[6652  410]
 [1612  217]]
              precision    recall  f1-score   support

           0       0.80      0.94      0.87      7062
           1       0.35      0.12      0.18      1829

    accuracy                           0.77      8891
   macro avg       0.58      0.53      0.52      8891
weighted avg       0.71      0.77      0.73      8891



In [168]:
o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

[[7062    0]
 [1828    1]]
              precision    recall  f1-score   support

           0       0.79      1.00      0.89      7062
           1       1.00      0.00      0.00      1829

    accuracy                           0.79      8891
   macro avg       0.90      0.50      0.44      8891
weighted avg       0.84      0.79      0.70      8891



In [171]:
o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

[[6940  122]
 [1731   98]]
              precision    recall  f1-score   support

           0       0.80      0.98      0.88      7062
           1       0.45      0.05      0.10      1829

    accuracy                           0.79      8891
   macro avg       0.62      0.52      0.49      8891
weighted avg       0.73      0.79      0.72      8891



In [173]:
o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

[[6718  344]
 [1714  115]]
              precision    recall  f1-score   support

           0       0.80      0.95      0.87      7062
           1       0.25      0.06      0.10      1829

    accuracy                           0.77      8891
   macro avg       0.52      0.51      0.48      8891
weighted avg       0.68      0.77      0.71      8891



In [149]:



# p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np,test_eps_lstm_np, test_eps_vec_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

completed------5 

[[7059    3]
 [1829    0]]
              precision    recall  f1-score   support

           0       0.79      1.00      0.89      7062
           1       0.00      0.00      0.00      1829

    accuracy                           0.79      8891
   macro avg       0.40      0.50      0.44      8891
weighted avg       0.63      0.79      0.70      8891



In [137]:
X_train_df

,Adj_YoY,Adj_surprise,Adj_YoYRev,RSI9,RSI14,RSI20,VWMA9_ch_p,VWMA14_ch_p,VWMA20_ch_p,1d_ch_p,2d_ch_p,3d_ch_p,7d_ch_p
0,0.005407,0.052908,0.861154,0.531952,0.573029,0.616529,0.070330,0.162753,0.220182,0.007254,0.009108,0.034722,0.193288
1,0.003673,0.011859,1.419465,0.722054,0.716000,0.736283,0.048115,0.109316,0.140843,-0.052126,-0.025432,-0.050705,0.089964
2,0.000291,0.039272,0.121790,1.192702,1.113842,1.053255,-0.103260,-0.102613,-0.116800,0.054980,-0.035961,-0.083522,-0.168799
3,0.127379,0.136322,1.733934,0.932742,1.054986,1.110072,0.195228,0.109557,0.056114,0.168044,0.456292,0.452385,-0.048191
4,0.004134,0.091524,0.968236,1.128751,1.087713,1.065031,-0.036381,-0.058686,-0.000090,-0.046430,-0.053739,-0.004909,-0.046430
...,...,...,...,...,...,...,...,...,...,...,...,...,...
234900,0.005665,0.035620,0.048354,1.116727,0.948461,0.859905,-0.085264,-0.125786,-0.111490,-0.044928,-0.027317,-0.087844,-0.189712
234901,0.006366,0.490309,1.300834,1.254572,1.126924,1.064227,-0.186760,-0.241261,-0.200556,-0.077322,-0.275986,-0.244291,-0.354265
234902,0.000018,0.000000,1.216629,0.435016,0.614554,0.733274,0.179758,0.285736,0.420829,0.016134,-0.005372,0.055360,0.400782
234903,0.000021,0.011080,0.318248,1.083885,1.139766,1.153376,0.029223,0.051027,-0.044391,-0.075438,0.149390,0.135144,-0.006116


In [135]:
X_train_df

,Adj_YoY,Adj_surprise,Adj_YoYRev,RSI9,RSI14,RSI20,VWMA9_ch_p,VWMA14_ch_p,VWMA20_ch_p,1d_ch_p,2d_ch_p,3d_ch_p,7d_ch_p
0,0.005407,0.052908,0.861154,1.989361,1.988539,1.987669,0.070330,0.162753,0.220182,0.007254,0.009108,0.034722,0.193288
1,0.003673,0.011859,1.419465,1.985559,1.985680,1.985274,0.048115,0.109316,0.140843,-0.052126,-0.025432,-0.050705,0.089964
2,0.000291,0.039272,0.121790,1.976146,1.977723,1.978935,-0.103260,-0.102613,-0.116800,0.054980,-0.035961,-0.083522,-0.168799
3,0.127379,0.136322,1.733934,1.981345,1.978900,1.977799,0.195228,0.109557,0.056114,0.168044,0.456292,0.452385,-0.048191
4,0.004134,0.091524,0.968236,1.977425,1.978246,1.978699,-0.036381,-0.058686,-0.000090,-0.046430,-0.053739,-0.004909,-0.046430
...,...,...,...,...,...,...,...,...,...,...,...,...,...
234900,0.005665,0.035620,0.048354,1.977665,1.981031,1.982802,-0.085264,-0.125786,-0.111490,-0.044928,-0.027317,-0.087844,-0.189712
234901,0.006366,0.490309,1.300834,1.974909,1.977462,1.978715,-0.186760,-0.241261,-0.200556,-0.077322,-0.275986,-0.244291,-0.354265
234902,0.000018,0.000000,1.216629,1.991300,1.987709,1.985335,0.179758,0.285736,0.420829,0.016134,-0.005372,0.055360,0.400782
234903,0.000021,0.011080,0.318248,1.978322,1.977205,1.976932,0.029223,0.051027,-0.044391,-0.075438,0.149390,0.135144,-0.006116


In [139]:
def vola(x):
    if x >= 0.02:
        return 1
    elif x <= -0.02:
        return 1
    else:
        return 0

new_df['vol1'] = new_df["fut_snp_5d_ch_p"].apply(vola)
test_df['vol1'] = test_df["fut_snp_5d_ch_p"].apply(vola)

In [140]:
new_df['vol'].value_counts(normalize = True)

0    0.765173
1    0.234827
Name: vol, dtype: float64

In [141]:
test_df['vol1'].value_counts(normalize = True)

0    0.624227
1    0.375773
Name: vol1, dtype: float64

In [142]:
p = model.predict([ test_pattern_lstm_np,test_volume_lstm_np, X_test_np,test_eps_lstm_np, test_eps_vec_np])

print('completed------5 \n')

o = []
for i in p:
    o.append(i.argmax())
    
print(confusion_matrix(test_label_np.tolist(), o))

print(classification_report(test_label_np.tolist(), o))

completed------5 

[[6857  205]
 [1697  132]]
              precision    recall  f1-score   support

           0       0.80      0.97      0.88      7062
           1       0.39      0.07      0.12      1829

    accuracy                           0.79      8891
   macro avg       0.60      0.52      0.50      8891
weighted avg       0.72      0.79      0.72      8891



In [120]:
p

array([[0.9320192 , 0.06798082],
       [0.9258238 , 0.07417624],
       [0.9309752 , 0.06902475],
       ...,
       [0.91011477, 0.08988518],
       [0.9137494 , 0.08625058],
       [0.90693605, 0.09306397]], dtype=float32)

In [127]:
o = []
for i in p[:,1].tolist():
    if i  > 0.18:
        o.append(1)
    else:
        o.append(0)
    
print(confusion_matrix(test_df['vol'].tolist(), o))

print(classification_report(test_df['vol'].tolist(), o))

[[6972   90]
 [1754   75]]
              precision    recall  f1-score   support

           0       0.80      0.99      0.88      7062
           1       0.45      0.04      0.08      1829

    accuracy                           0.79      8891
   macro avg       0.63      0.51      0.48      8891
weighted avg       0.73      0.79      0.72      8891



In [51]:
print('              precision    recall  f1-score   support\n\n           0       0.52      0.95      0.68      4587\n           1       0.60      0.08      0.14      4304\n\n    accuracy                           0.53      8891\n   macro avg       0.56      0.52      0.41      8891\nweighted avg       0.56      0.53      0.42      8891\n')

              precision    recall  f1-score   support

           0       0.52      0.95      0.68      4587
           1       0.60      0.08      0.14      4304

    accuracy                           0.53      8891
   macro avg       0.56      0.52      0.41      8891
weighted avg       0.56      0.53      0.42      8891



In [52]:
X_train6_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p','1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p',
                     '1d_adx9_ch_p', '2d_adx9_ch_p', '3d_adx9_ch_p', '7d_adx9_ch_p',
                      '1d_adx14_ch_p', '2d_adx14_ch_p', '3d_adx14_ch_p', '7d_adx14_ch_p',
                     '1d_RSI14_ch_p', '2d_RSI14_ch_p', '3d_RSI14_ch_p', '7d_RSI14_ch_p',"VWMA2_ch_p","VWMA5_ch_p","VWMA30_ch_p",'1d_ch_v','7d_ch_v','12d_ch_v','21d_ch_v',
       'vola5','vola9','vola21','macd259','signal259','hist259','macd91121','signal91121','hist91121','1d_DI+14_ushift',
                      '2d_DI+14_ushift','3d_DI+14_ushift','7d_DI+14_ushift','1d_DI-14_ushift','2d_DI-14_ushift',
                      '3d_DI-14_ushift','7d_DI-14_ushift','1d_DI+20_ushift','2d_DI+20_ushift',
                      '3d_DI+20_ushift','7d_DI+20_ushift','1d_DI-20_ushift','2d_DI-20_ushift',
                      '3d_DI-20_ushift','7d_DI-20_ushift']]
# X_train_df = pd.concat([X_train0_df[:train_len], X_train1_df[:train_len],  X_train2_df[:train_len], X_train6_df[:train_len]], axis=1)
# X_test_df = pd.concat([X_train0_df[train_len:], X_train1_df[train_len:],  X_train2_df[train_len:], X_train3_df[train_len:]], axis=1)

X_train_np = np.array(X_train6_df[:train_len])
X_test_np = np.array(X_train6_df[train_len:])

In [53]:
from catboost import CatBoostClassifier, Pool

X_train_np_, X_val_np, label_np_, val_label_np = train_test_split(X_train_np, label_np, test_size=0.2)
test_data =  Pool(X_val_np, val_label_np)

model = CatBoostClassifier(
                            )

# model = CatBoostClassifier(
#                             )
# model = CatBoostClassifier(iterations=1000,
#                         depth=9,
#                         learning_rate=0.1,
#                         # loss_function='Logloss',
#                         eval_metric='AUC',
#                         )
# train the model
model.fit(X_train_np_, label_np_, eval_set = test_data, use_best_model=True, early_stopping_rounds=100)

Learning rate set to 0.1155
0:	learn: 0.6477733	test: 0.6474890	best: 0.6474890 (0)	total: 76.8ms	remaining: 1m 16s
1:	learn: 0.6133018	test: 0.6126958	best: 0.6126958 (1)	total: 101ms	remaining: 50.3s
2:	learn: 0.5871383	test: 0.5863651	best: 0.5863651 (2)	total: 124ms	remaining: 41.2s
3:	learn: 0.5661681	test: 0.5652539	best: 0.5652539 (3)	total: 147ms	remaining: 36.5s
4:	learn: 0.5509038	test: 0.5499100	best: 0.5499100 (4)	total: 171ms	remaining: 34.1s
5:	learn: 0.5396316	test: 0.5385499	best: 0.5385499 (5)	total: 194ms	remaining: 32.2s
6:	learn: 0.5299428	test: 0.5288010	best: 0.5288010 (6)	total: 219ms	remaining: 31.1s
7:	learn: 0.5230288	test: 0.5218823	best: 0.5218823 (7)	total: 241ms	remaining: 29.9s
8:	learn: 0.5176593	test: 0.5165279	best: 0.5165279 (8)	total: 266ms	remaining: 29.2s
9:	learn: 0.5128567	test: 0.5116829	best: 0.5116829 (9)	total: 288ms	remaining: 28.5s
10:	learn: 0.5093559	test: 0.5082249	best: 0.5082249 (10)	total: 311ms	remaining: 28s
11:	learn: 0.5064351	tes

In [55]:
eval_data = X_test_np
preds_proba = model.predict_proba(eval_data)
# preds_class = model.predict(test_data)
pr_d = pd.DataFrame({'val':preds_proba[:,1].tolist()})
pr_d['pred'] = pr_d['val'].apply(lambda x: 1 if x >= 0.4 else 0)
pr_d['Date'] = test_df['Date']
o_d = []
for i in pr_d['pred'].tolist():
#     if i[1] > 0.7:''
#         o.append(1)z
#     else:
#         o.append(o)
    
    o_d.append(i)
    
print(confusion_matrix(test_label_np.tolist(), o_d))

print(classification_report(test_label_np.tolist(), o_d))

[[6931  131]
 [1716  113]]
              precision    recall  f1-score   support

           0       0.80      0.98      0.88      7062
           1       0.46      0.06      0.11      1829

    accuracy                           0.79      8891
   macro avg       0.63      0.52      0.50      8891
weighted avg       0.73      0.79      0.72      8891



In [3]:
stock_df = pd.read_csv('historical_data/AAPL.csv')

In [4]:
stock_df["Adj_open"] = stock_df["Open"] * stock_df["Adjusted_close"] / stock_df["Close"]
stock_df["Adj_high"] = stock_df["High"] * stock_df["Adjusted_close"] / stock_df["Close"]
stock_df["Adj_low"] = stock_df["Low"] * stock_df["Adjusted_close"] / stock_df["Close"]
stock_df["Adj_volume"] = stock_df["Volume"] * stock_df["Close"] / stock_df["Adjusted_close"]
stock_df["Tx_amount"] = stock_df["Volume"] * stock_df["Close"]

In [5]:
stock_sel_df = stock_df[['Date', 'Adjusted_close', 'Adj_open', 'Adj_high', 'Adj_low', 'Adj_volume', 'Close', 'Volume', 'Tx_amount']]
print(stock_sel_df)

             Date  Adjusted_close  Adj_open    Adj_high   Adj_low  \
0      1980-12-11          0.0769    0.0769    0.076900    0.0769   
1      1980-12-12          0.1006    0.1006    0.101041    0.1006   
2      1980-12-15          0.0954    0.0954    0.095841    0.0954   
3      1980-12-16          0.0883    0.0883    0.088741    0.0883   
4      1980-12-17          0.0905    0.0905    0.090934    0.0905   
...           ...             ...       ...         ...       ...   
10215  2021-06-21        132.3000  130.3000  132.410000  129.2100   
10216  2021-06-22        133.9800  132.1300  134.080000  131.6200   
10217  2021-06-23        133.7000  133.7700  134.320000  133.2300   
10218  2021-06-24        133.4100  134.4500  134.640000  132.9300   
10219  2021-06-25        133.1100  133.4600  133.890000  132.8100   

         Adj_volume    Close    Volume     Tx_amount  
0      0.000000e+00   22.000         0  0.000000e+00  
1      5.984058e+08   28.750   2093900  6.019962e+07  
2     

In [6]:
from datetime import datetime

def addEpoch(date):
    epoch = datetime.strptime(date, "%Y-%m-%d").timestamp()
    return epoch

In [7]:
# add epoch to data frame
stock_sel_df["Epoch"]=stock_sel_df["Date"].apply(addEpoch)
print(stock_sel_df)

             Date  Adjusted_close  Adj_open    Adj_high   Adj_low  \
0      1980-12-11          0.0769    0.0769    0.076900    0.0769   
1      1980-12-12          0.1006    0.1006    0.101041    0.1006   
2      1980-12-15          0.0954    0.0954    0.095841    0.0954   
3      1980-12-16          0.0883    0.0883    0.088741    0.0883   
4      1980-12-17          0.0905    0.0905    0.090934    0.0905   
...           ...             ...       ...         ...       ...   
10215  2021-06-21        132.3000  130.3000  132.410000  129.2100   
10216  2021-06-22        133.9800  132.1300  134.080000  131.6200   
10217  2021-06-23        133.7000  133.7700  134.320000  133.2300   
10218  2021-06-24        133.4100  134.4500  134.640000  132.9300   
10219  2021-06-25        133.1100  133.4600  133.890000  132.8100   

         Adj_volume    Close    Volume     Tx_amount         Epoch  
0      0.000000e+00   22.000         0  0.000000e+00  3.453588e+08  
1      5.984058e+08   28.750   20

In [8]:
#Start working on subset

In [9]:
from_date = "2010-11-15" 
from_epoch = datetime.strptime(from_date, "%Y-%m-%d").timestamp()

to_date = "2020-02-15" 
to_epoch = datetime.strptime(to_date, "%Y-%m-%d").timestamp()

In [10]:
# selecting rows based on condition
new_df = stock_sel_df.loc[(stock_sel_df['Epoch'] >= from_epoch) & (stock_sel_df['Epoch'] <= to_epoch)]
print(new_df)

            Date  Adjusted_close   Adj_open   Adj_high    Adj_low  \
7549  2010-11-15          9.4294   9.473163   9.537043   9.405906   
7550  2010-11-16          9.2622   9.389037   9.446774   9.192486   
7551  2010-11-17          9.2288   9.250298   9.335983   9.144651   
7552  2010-11-18          9.4723   9.373102   9.510382   9.357440   
7553  2010-11-19          9.4201   9.458182   9.471388   9.374340   
...          ...             ...        ...        ...        ...   
9872  2020-02-10         79.6339  77.808673  79.633900  77.726946   
9873  2020-02-11         79.1534  80.141548  80.215845  78.930509   
9874  2020-02-12         81.0332  79.614128  81.038153  79.614128   
9875  2020-02-13         80.4561  80.287694  80.790436  80.079662   
9876  2020-02-14         80.4759  80.423892  80.730986  79.955822   

        Adj_volume    Close    Volume     Tx_amount         Epoch  
7549  4.693546e+08  307.035  14414422  4.425732e+09  1.289797e+09  
7550  7.662964e+08  301.590  235339

In [11]:
#Complete creating subset from 2014-11-15 to 2020-01-15 and Epoch

In [12]:
def addUpIndicator(change_p):
    if change_p > 0:
        return 'U'
    else:
        return 'D'

In [13]:
def add3LIndicator(change_p):
    if change_p >= 0.005:
        return 'U'
    elif change_p <= -0.005:
        return 'D'
    else:
        return 'N'

In [14]:
#Calculate change in pricing n days in the future
shift = "nd_dshift"
change_p = "future_ch_p"
up = "up"
l3 = "3l"
n=1

In [15]:
new_df[shift] = new_df["Adjusted_close"]
new_df[shift] = new_df[shift].shift(-n)
new_df = new_df[:-n]

new_df[change_p] = (new_df[shift]-new_df["Adjusted_close"])/new_df["Adjusted_close"]

new_df[up]=new_df[change_p].apply(addUpIndicator)
new_df[l3]=new_df[change_p].apply(add3LIndicator)

In [16]:
#Calculate change in Pricing & Volume relative to 1 day in the past
shift = "1ushift"
shift_v = "1ushift_v"
change_p = "past1d_ch_p"
change_v = "past1d_ch_v"

In [17]:
new_df[shift] = new_df["Adjusted_close"]
new_df[shift] = new_df[shift].shift(1)

new_df[shift_v] = new_df["Volume"]
new_df[shift_v] = new_df[shift_v].shift(1)

new_df = new_df[1:]

new_df[change_p] = (new_df["Adjusted_close"] - new_df[shift])/new_df[shift]
new_df[change_v] = (new_df["Volume"] - new_df[shift_v])/new_df[shift_v]

In [18]:
# Calculate Directional Volume (similar to On Balance Volume)
new_df["Directional_volume"] = np.sign(new_df[change_p]) * new_df["Volume"]

In [19]:
new_df["up"].value_counts(normalize=True)

U    0.527945
D    0.472055
Name: up, dtype: float64

In [20]:
new_df["3l"].value_counts(normalize=True)

U    0.378332
N    0.323732
D    0.297936
Name: 3l, dtype: float64

In [21]:
import numpy as np

def addLabel(value):
    if value == 'U':
        a = 2
    elif value == 'N':
        a = 1
    elif value == 'D':
        a = 0
        
    return a

In [22]:
new_df["label"] = new_df["3l"].apply(addLabel)

In [23]:
#Completed labeling for predicting n days out
#Start normalizing input data

In [24]:
#from sklearn.preprocessing import MinMaxScaler
#sc = MinMaxScaler(feature_range = (0, 1))
#training_set_scaled = sc.fit_transform(training_set)

In [25]:
max_close = new_df["Adjusted_close"].max()
new_df["n_Adjusted_close"] = new_df["Adjusted_close"]/max_close
#new_df["n_Adjusted_close"] = new_df["Adjusted_close"]
new_df["n_Adjusted_open"] = new_df["Adj_open"]/max_close
#new_df["n_Adjusted_open"] = new_df["Adj_open"]
new_df["n_Adjusted_high"] = new_df["Adj_high"]/max_close
#new_df["n_Adjusted_high"] = new_df["Adj_high"]
new_df["n_Adjusted_low"] = new_df["Adj_low"]/max_close
#new_df["n_Adjusted_low"] = new_df["Adj_low"]
new_df["n_Adjusted_volume"] = new_df["Adj_volume"]/new_df["Adj_volume"].max()
#new_df["n_Adjusted_volume"] = new_df["Adj_volume"]
#new_df["n_Tx_amount"] = new_df["Tx_amount"]/new_df["Tx_amount"].max()
max_dir_vol = new_df["Directional_volume"].max()
min_dir_vol = new_df["Directional_volume"].min()
n_factor = max(max_dir_vol, -min_dir_vol)
new_df["n_Directional_volume"] = new_df["Directional_volume"]/n_factor
print(max_dir_vol, min_dir_vol, n_factor)
print(new_df)

146477063.0 -189846255.0 189846255.0
            Date  Adjusted_close   Adj_open   Adj_high    Adj_low  \
7550  2010-11-16          9.2622   9.389037   9.446774   9.192486   
7551  2010-11-17          9.2288   9.250298   9.335983   9.144651   
7552  2010-11-18          9.4723   9.373102   9.510382   9.357440   
7553  2010-11-19          9.4201   9.458182   9.471388   9.374340   
7554  2010-11-22          9.6236   9.418450   9.623600   9.393575   
...          ...             ...        ...        ...        ...   
9871  2020-02-07         79.2575  79.837016  80.092102  78.754757   
9872  2020-02-10         79.6339  77.808673  79.633900  77.726946   
9873  2020-02-11         79.1534  80.141548  80.215845  78.930509   
9874  2020-02-12         81.0332  79.614128  81.038153  79.614128   
9875  2020-02-13         80.4561  80.287694  80.790436  80.079662   

        Adj_volume   Close    Volume     Tx_amount         Epoch  ...  \
7550  7.662964e+08  301.59  23533906  7.097591e+09  1.289884e

In [26]:
#price_df = new_df[["n_Adjusted_close", "n_Adjusted_open", "n_Adjusted_high", "n_Adjusted_low", "n_Adjusted_volume", "n_Tx_amount"]]
price_df = new_df[["n_Adjusted_close", "n_Adjusted_open", "n_Adjusted_high", "n_Adjusted_low"]]
volume_df = new_df[["n_Adjusted_volume", "n_Directional_volume"]]
#us_price_df = new_df[["n_Adjusted_close", "n_Adjusted_open", "n_Adjusted_high", "n_Adjusted_low", "n_Adjusted_volume"]]
#price_df = sc.fit_transform(us_price_df)

In [27]:
lstm_seq_size = 40
n = lstm_seq_size
price_lstm_ = []
volume_lstm_ = []

for i in range(n-1, price_df.shape[0]):
    price_lstm_.append(price_df[i-n+1:i+1])
    volume_lstm_.append(volume_df[i-n+1:i+1])

In [28]:
price_lstm_np = np.array(price_lstm_)
volume_lstm_np = np.array(volume_lstm_)

In [29]:
print(price_lstm_np)

[[[0.1143013  0.11586655 0.11657906 0.11344098]
  [0.11388912 0.11415442 0.11521183 0.11285067]
  [0.11689406 0.1156699  0.11736402 0.11547661]
  ...
  [0.12978878 0.12841492 0.1300825  0.12778579]
  [0.12948026 0.13070821 0.13073853 0.12865784]
  [0.13053292 0.1300895  0.13053671 0.12961575]]

 [[0.11388912 0.11415442 0.11521183 0.11285067]
  [0.11689406 0.1156699  0.11736402 0.11547661]
  [0.11624988 0.11671984 0.11688281 0.11568518]
  ...
  [0.12948026 0.13070821 0.13073853 0.12865784]
  [0.13053292 0.1300895  0.13053671 0.12961575]
  [0.1310105  0.13081342 0.13137433 0.13031694]]

 [[0.11689406 0.1156699  0.11736402 0.11547661]
  [0.11624988 0.11671984 0.11688281 0.11568518]
  [0.1187612  0.11622953 0.1187612  0.11592254]
  ...
  [0.13053292 0.1300895  0.13053671 0.12961575]
  [0.1310105  0.13081342 0.13137433 0.13031694]
  [0.13207179 0.1310902  0.13207179 0.13054066]]

 ...

 [[0.83893145 0.82768065 0.8393888  0.82606468]
  [0.85329223 0.8445721  0.8561278  0.84451112]
  [0.85496

In [30]:
print(volume_lstm_np)

[[[ 0.35032175 -0.12396297]
  [ 0.254891   -0.09019509]
  [ 0.262888    0.09302448]
  ...
  [ 0.23847024  0.08438368]
  [ 0.23608892 -0.08354128]
  [ 0.16086215  0.05692148]]

 [[ 0.254891   -0.09019509]
  [ 0.262888    0.09302448]
  [ 0.20459584 -0.07239753]
  ...
  [ 0.23608892 -0.08354128]
  [ 0.16086215  0.05692148]
  [ 0.15850558  0.05608763]]

 [[ 0.262888    0.09302448]
  [ 0.20459584 -0.07239753]
  [ 0.20897414  0.07394592]
  ...
  [ 0.16086215  0.05692148]
  [ 0.15850558  0.05608763]
  [ 0.16419094  0.05809946]]

 ...

 [[ 0.06186189  0.17610464]
  [ 0.05936079  0.16898466]
  [ 0.05287479  0.15052074]
  ...
  [ 0.0543098  -0.15497283]
  [ 0.05046324  0.1439966 ]
  [ 0.04352906 -0.12420988]]

 [[ 0.05936079  0.16898466]
  [ 0.05287479  0.15052074]
  [ 0.05370536 -0.15288522]
  ...
  [ 0.05046324  0.1439966 ]
  [ 0.04352906 -0.12420988]
  [ 0.05248519  0.14976631]]

 [[ 0.05287479  0.15052074]
  [ 0.05370536 -0.15288522]
  [ 0.04556812  0.12972048]
  ...
  [ 0.04352906 -0.124209

In [31]:
print(price_lstm_np.shape)
print(volume_lstm_np.shape)

(2287, 40, 4)
(2287, 40, 2)


In [32]:
#Regular MLP input

In [33]:
new_df["1d_ushift"] = new_df["Adjusted_close"]
new_df["1d_ushift"] = new_df["1d_ushift"].shift(1)

new_df["2d_ushift"] = new_df["Adjusted_close"]
new_df["2d_ushift"] = new_df["2d_ushift"].shift(2)

new_df["3d_ushift"] = new_df["Adjusted_close"]
new_df["3d_ushift"] = new_df["3d_ushift"].shift(3)

new_df["7d_ushift"] = new_df["Adjusted_close"]
new_df["7d_ushift"] = new_df["7d_ushift"].shift(7)

In [38]:
new_df["RSI9"] = np.nan
new_df["RSI14"] = np.nan
new_df["RSI20"] = np.nan
new_df["VWMA9"] = np.nan
new_df["VWMA14"] = np.nan
new_df["VWMA20"] = np.nan

In [39]:
import tulipy as ti

In [40]:
ti_periods = [9, 14, 20]

In [41]:
for n in ti_periods:
    rsi_calc = ti.rsi(new_df["Adjusted_close"].to_numpy(), period=n)
    vwma_calc = ti.vwma(new_df["Adjusted_close"].astype(float).to_numpy(), new_df["Volume"].astype(float).to_numpy(), period=n)
    print(rsi_calc)
    print(vwma_calc)
    for i in range(n-1, new_df.shape[0]-1):
        col_name = "RSI" + str(n)
        new_df.loc[new_df.index[i], col_name] = rsi_calc[i-n+1]
        col_name = "VWMA" + str(n)
        new_df.loc[new_df.index[i], col_name] = vwma_calc[i-n+1]

[63.34060342 68.52740394 70.11499043 ... 55.37596415 63.79529547
 59.89252231]
[ 9.48198674  9.52897072  9.58885274 ... 78.5126666  78.59457488
 79.15839775]
[67.24778701 69.17677215 67.26193808 ... 58.29990951 63.66434421
 61.06706794]
[ 9.56651114  9.61355179  9.65879919 ... 78.53888066 78.66434428
 78.75370602]
[66.87551236 67.42564295 66.59287065 ... 60.67850244 64.47452992
 62.52395967]
[ 9.63328779  9.67422597  9.70903836 ... 78.35912893 78.54039914
 78.68374265]


In [42]:
print(new_df)

            Date  Adjusted_close   Adj_open   Adj_high    Adj_low  \
7550  2010-11-16          9.2622   9.389037   9.446774   9.192486   
7551  2010-11-17          9.2288   9.250298   9.335983   9.144651   
7552  2010-11-18          9.4723   9.373102   9.510382   9.357440   
7553  2010-11-19          9.4201   9.458182   9.471388   9.374340   
7554  2010-11-22          9.6236   9.418450   9.623600   9.393575   
...          ...             ...        ...        ...        ...   
9871  2020-02-07         79.2575  79.837016  80.092102  78.754757   
9872  2020-02-10         79.6339  77.808673  79.633900  77.726946   
9873  2020-02-11         79.1534  80.141548  80.215845  78.930509   
9874  2020-02-12         81.0332  79.614128  81.038153  79.614128   
9875  2020-02-13         80.4561  80.287694  80.790436  80.079662   

        Adj_volume   Close    Volume     Tx_amount         Epoch  ...  \
7550  7.662964e+08  301.59  23533906  7.097591e+09  1.289884e+09  ...   
7551  5.575505e+08  300.5

In [43]:
new_df[:30]

,Date,Adjusted_close,Adj_open,Adj_high,Adj_low,Adj_volume,Close,Volume,Tx_amount,Epoch,...,1d_ushift,2d_ushift,3d_ushift,7d_ushift,RSI9,RSI14,RSI20,VWMA9,VWMA14,VWMA20
7550,2010-11-16,9.2622,9.389037,9.446774,9.192486,7.662964e+08,301.5900,23533906,7.097591e+09,1.289884e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7551,2010-11-17,9.2288,9.250298,9.335983,9.144651,5.575505e+08,300.5000,17123201,5.145522e+09,1.289970e+09,...,9.2622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7552,2010-11-18,9.4723,9.373102,9.510382,9.357440,5.750432e+08,308.4300,17660349,5.446981e+09,1.290056e+09,...,9.2288,9.2622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7553,2010-11-19,9.4201,9.458182,9.471388,9.374340,4.475345e+08,306.7300,13744399,4.215820e+09,1.290143e+09,...,9.4723,9.2288,9.2622,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7554,2010-11-22,9.6236,9.418450,9.623600,9.393575,4.571116e+08,313.3600,14038356,4.399059e+09,1.290402e+09,...,9.4201,9.4723,9.2288,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7555,2010-11-23,9.4815,9.534323,9.574248,9.414856,6.040647e+08,308.7300,18551612,5.727439e+09,1.290488e+09,...,9.6236,9.4201,9.4723,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7556,2010-11-24,9.6678,9.581962,9.686380,9.574284,4.817573e+08,314.7950,14795448,4.657533e+09,1.290575e+09,...,9.4815,9.6236,9.4201,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7557,2010-11-26,9.6974,9.635363,9.756977,9.610794,2.764745e+08,315.7600,8490891,2.681084e+09,1.290748e+09,...,9.6678,9.4815,9.6236,9.2622,NaN,NaN,NaN,NaN,NaN,NaN
7558,2010-11-29,9.7315,9.689425,9.750234,9.562895,5.183749e+08,316.8700,15919985,5.044566e+09,1.291007e+09,...,9.6974,9.6678,9.4815,9.2288,63.340603,NaN,NaN,9.481987,NaN,NaN
7559,2010-11-30,9.5558,9.629200,9.654383,9.547201,5.836083e+08,311.1500,17923329,5.576844e+09,1.291093e+09,...,9.7315,9.6974,9.6678,9.4723,68.527404,NaN,NaN,9.528971,NaN,NaN


In [44]:
print(new_df["future_ch_p"], new_df["nd_dshift"])

7550   -0.003606
7551    0.026385
7552   -0.005511
7553    0.021603
7554   -0.014766
          ...   
9871    0.004749
9872   -0.006034
9873    0.023749
9874   -0.007122
9875    0.000246
Name: future_ch_p, Length: 2326, dtype: float64 7550     9.2288
7551     9.4723
7552     9.4201
7553     9.6236
7554     9.4815
         ...   
9871    79.6339
9872    79.1534
9873    81.0332
9874    80.4561
9875    80.4759
Name: nd_dshift, Length: 2326, dtype: float64


In [45]:
from_date = "2011-01-01" 
from_epoch = datetime.strptime(from_date, "%Y-%m-%d").timestamp()

# selecting rows based on condition
new_df = new_df.loc[new_df['Epoch'] >= from_epoch]
new_df = new_df[lstm_seq_size-2:-1]
print(new_df)

            Date  Adjusted_close   Adj_open   Adj_high    Adj_low  \
7620  2011-02-28         10.8476  10.787098  10.904109  10.783413   
7621  2011-03-01         10.7278  10.916982  10.924660  10.677771   
7622  2011-03-02         10.8140  10.747664  10.882486  10.699755   
7623  2011-03-03         11.0425  10.969715  11.049564  10.930711   
7624  2011-03-04         11.0561  11.058250  11.065006  10.986999   
...          ...             ...        ...        ...        ...   
9870  2020-02-06         80.3496  79.697335  80.352071  79.127790   
9871  2020-02-07         79.2575  79.837016  80.092102  78.754757   
9872  2020-02-10         79.6339  77.808673  79.633900  77.726946   
9873  2020-02-11         79.1534  80.141548  80.215845  78.930509   
9874  2020-02-12         81.0332  79.614128  81.038153  79.614128   

        Adj_volume   Close    Volume     Tx_amount         Epoch  ...  \
7620  4.687335e+08  353.21  14395497  5.084633e+09  1.298869e+09  ...   
7621  5.304319e+08  349.3

In [46]:
new_df["VWMA9_ch_p"] = (new_df["Adjusted_close"]-new_df["VWMA9"])/new_df["VWMA9"]
new_df["VWMA14_ch_p"] = (new_df["Adjusted_close"]-new_df["VWMA14"])/new_df["VWMA14"]
new_df["VWMA20_ch_p"] = (new_df["Adjusted_close"]-new_df["VWMA20"])/new_df["VWMA20"]

In [47]:
new_df["1d_ch_p"] = (new_df["Adjusted_close"]-new_df["1d_ushift"])/new_df["1d_ushift"]
new_df["2d_ch_p"] = (new_df["Adjusted_close"]-new_df["2d_ushift"])/new_df["2d_ushift"]
new_df["3d_ch_p"] = (new_df["Adjusted_close"]-new_df["3d_ushift"])/new_df["3d_ushift"]
new_df["7d_ch_p"] = (new_df["Adjusted_close"]-new_df["7d_ushift"])/new_df["7d_ushift"]

In [48]:
new_df.shape

(2255, 43)

In [ ]:
X_train1_df = (new_df[['RSI9', 'RSI14', 'RSI20']] - 50)/50
X_train2_df = new_df[['VWMA9_ch_p', 'VWMA14_ch_p', 'VWMA20_ch_p']] * 10
X_train3_df = new_df[['1d_ch_p', '2d_ch_p', '3d_ch_p', '7d_ch_p']] * 10
X_train_df = pd.concat([X_train1_df, X_train2_df, X_train3_df], axis=1)
print(X_train_df)
X_train_np = X_train_df.to_numpy()
print(X_train_np)

In [ ]:
#Get label np

In [ ]:
label_df = new_df["label"]

In [ ]:
print(label_df)

In [ ]:
label_np = label_df.to_numpy()

In [ ]:
print(label_np)
print(label_np.shape)

In [ ]:
#Get change np

In [ ]:
change_df = new_df["future_ch_p"] * 50
change_df = change_df[lstm_seq_size-1:]

In [ ]:
print(change_df)

In [ ]:
change_np = change_df.to_numpy()

In [ ]:
print(change_np)
print(change_np.shape)

In [ ]:
#Model multi input and mixed data

In [ ]:
# example of a model defined with the sequential api
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense, Conv2D, Flatten, concatenate
from keras.layers.recurrent import LSTM
from keras.layers import Dropout, Bidirectional

In [ ]:
# define two sets of inputs
price = Input(shape=(lstm_seq_size,4))
volume = Input(shape=(lstm_seq_size,2))
technicals = Input(shape=(10,))
# the first branch operates on the first input - price
x = LSTM(50,input_shape = (lstm_seq_size,4),return_sequences = True, activation = 'relu')(price)
x = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(x)
x = Dropout(0.1)(x)
x = Model(inputs=price, outputs=x)
# the second branch opreates on the second input - volume
y = LSTM(50,input_shape = (lstm_seq_size,2),return_sequences = True, activation = 'relu')(volume)
y = Bidirectional(LSTM(60,return_sequences = False, activation = 'relu'))(y)
y = Dropout(0.1)(y)
y = Model(inputs=volume, outputs=y)
# the third branch opreates on the third input - technicals
z = Dense(60, activation='relu', kernel_initializer='he_normal', input_shape=(10,))(technicals)
z = Dense(30, activation='relu')(z)
z = Dropout(0.1)(z)
z = Model(inputs=techicals, outputs=z)
# combine the output of the two branches
combined = concatenate([x.output, y.output, z.output])
combined = Dropout(0.1)(combined)
#combined = Multiply()([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
out = Dense(3)(combined)
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input, z.input], outputs=out)

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# compile the model
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn, metrics=['accuracy'])

In [ ]:
# fit the model
model.fit([price_lstm_np, volume_lstm_np, X_train_np], label_np, epochs=150, batch_size=64, verbose=1)
print(model.summary())

In [ ]:
# define the model
model = Sequential()
model.add(Conv2D(64, 5, input_shape = (lstm_seq_size,6), activation = 'relu'))
model.add(Flatten())
model.add(Dense(3))

In [ ]:
# compile the model
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.1), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# fit the model
model.fit(price_lstm_np, label_np, epochs=50, batch_size=64, verbose=1)
print(model.summary())

In [ ]:
#Model an LSTM

In [ ]:
#from keras import optimizers
#define the model
model = Sequential()
model.add(Bidirectional(LSTM(50,input_shape = (lstm_seq_size,6),return_sequences = False, activation = 'relu')))
model.add(Dropout(.2))
model.add(Dense(50, activation = 'relu'))
model.add(Dropout(.2))
model.add(Dense(3))

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# compile the model
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = loss_fn,metrics=['accuracy'])

In [ ]:
# fit the model
model.fit(price_lstm_np, label_np, epochs=100, batch_size=64, verbose=1)
print(model.summary())

In [ ]:
# define two sets of inputs
inputA = Input(shape=(32,))
inputB = Input(shape=(128,))
# the first branch operates on the first input
x = Dense(8, activation="relu")(inputA)
x = Dense(4, activation="relu")(x)
x = Model(inputs=inputA, outputs=x)
# the second branch opreates on the second input
y = Dense(64, activation="relu")(inputB)
y = Dense(32, activation="relu")(y)
y = Dense(4, activation="relu")(y)
y = Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = concatenate([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(2, activation="relu")(combined)
z = Dense(1, activation="linear")(z)
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input], outputs=z)

In [ ]:
# evaluate the model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % acc)